In [26]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
#import requests
import pandas as pd
from skimage import morphology
from skimage.color import rgb2hsv
from scipy.spatial.distance import cdist
from scipy.stats.stats import mode
from PIL import Image
from collections import Counter
import cv2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
#import seaborn as sns

import concurrent.futures

/var/folders/8f/5fx_0z8s6bv_0tb_tmhs7pk00000gn/T/ipykernel_34159/911933074.py:10: DeprecationWarning: Please use `mode` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import mode


In [13]:
# from sklearn.datasets import load_files

# data_folder_path = '/Users/nuvanjanashashipraba/Desktop/Images'
# my_data = load_files(data_folder_path, categories =['folder1', 'folder2', 'folder3'])


In [66]:
import os
from PIL import Image

image = '/Users/nuvanjanashashipraba/Documents/GitHub/cherry2'

for filename in os.listdir(image):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(image, filename)
        img = Image.open(img_path)
    

In [28]:
def asymmetry_extraction(image):
    #01.read the image using the Matplotlib library 
    image = plt.imread(image)

    #02.Crop the image : this is done by finding the indices of the non zero elements(pixels with non zero intensity values)
    #in order to do that,here we use np.nonezero from the numpy library 
    #center of the  shape is the center of the image
    #the boarders of the shape are the boaders of the image
    y_nonzero, x_nonzero = np.nonzero(image)
    image = image[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)] #slice the image to include only the non zero regions

    #03.Cut the image in halves
    #04.Find the point of cutoff
    height, width = image.shape
    cutoff_of_width = width//2
    cutoff_of_height  = height // 2

    #05.Cut the image vertically and horizontally in two
    vertical1_im = image[:,:cutoff_of_width]
    vertical2_im = image[:,cutoff_of_width:]
    horizontal1_im = image[:cutoff_of_height,:]
    horizontal2_im = image[cutoff_of_height:,:]

    #Simply put, in step 03, 04 and 05, cuts the cropped image into four sections: top, bottom , left and right.
    #This is done by dividing the width and height of the image by 2 and using the resulting values to slice the image horizontally and vertically


    #06.Flip one of the images both vertically and horizontally to create mirror images for each of the four sections.
    # This is done by NumPy array indexing with negative  step size to reverse the order of elements along the specified axis. 
    vertical_indexer = [slice(None)]*vertical2_im.ndim
    horizontal_indexer = [slice(None)]*horizontal2_im.ndim
    #Here, slice(None) creates a slice object that includes all elements along the specified axis, and imVertical2.ndim and imHorizontal2.ndim retrun the number of dimensions of the respective arrays.
    #create two lists of slice objects with the same lenght as the number of dimesnions of the corresponding arrays.





    vertical_indexer[1]= slice(None,None, -1)
    vertical_indexer[0]=slice(None, None,-1)
    #modify the slice objects in the second position  of each list to include all elements alng the corresponding axis in reverse order, using the -1 step size.
    #This flips the array along that axis effectively



    vertical2_im = vertical2_im[tuple(vertical_indexer)]
    horizontal2_im = horizontal2_im[tuple(horizontal_indexer)]
    #convert each list of slice objects to tuple and use it for NumPy indexing to obtain the flipped images.



    #07. if the images don't have the same shape, cut the biggest image
    #This can happen if the shape of the original shape was an odd number
    vertical2_im =vertical2_im[0:vertical1_im.shape[0], 0:vertical1_im.shape[1]]
    horizontal2_im = horizontal2_im[0:horizontal1_im.shape[0], 0:horizontal1_im.shape[1]]

    im_bwx_vertical = cv2.bitwise_xor(vertical1_im,vertical2_im)
    im_bwx_horizontal = cv2.bitwise_xor(horizontal1_im, horizontal2_im)

    vertical_area = np.sum(im_bwx_vertical ==1)
    horizontal_area= np.sum(im_bwx_horizontal==1)
    mean_area = (vertical_area + horizontal_area) //2
    #np.sum is used to count the number of white pixels in each binary image im_bwx_vertical and im_bwx_horizontal. 
    #This count is performed using the logical expresion im_bwx_vertical == 1 and im_bwx_horizontal == 1 respectively. The expression returns a Boolean array that is True where the value of im_bwx_vertical or im_bwx_horizontal is equal to 1, and False elsewhere. The np.sum() function then counts the number of True values in the array and returns the total count of white pixels in each image.


    #Here, the asymmetry level is calculated as a percentage of the non zero pixels  in the overlapped image over the lesion area
    return(mean_area/np.sum(im==1))*100
    #np.sum(im==1) is used to count the toatal number of non zero( while) pixels in the original image
    #(mean_area/np.sum(im==1))*100 calculates the asymmetry level of the  input image as a percentage.
    #the numerator is the average area of asymmetry  between two halves of the input image, in pixels.
    # The denominator us the total number of non zero pixels in the input image.
    #The result is  then multiplied by 100  to convert the values in to percentage.






 




    



In [29]:
def perimeter_area(im):
    image = plt.imread(im)
    mask = image.copy() #creates a copy of the original image, which will be used to create a binary mask
    area = np.sum(mask) #calculates the area of the lesion by summing the values of all pixels in the binary mask
    struct_el = morphology.disk(1) #cretaes  a disk shaped structuring element with a radius of 1, which wil be used for binary erosion.
    #Binary erosion is a morphological operation used in image processing to reduce the size of foreground objects(usually represented as white pixels) and eliminate isolated pixels or small components
    #The operation is performed by sliding a small binary structuring element (also known as kernel or mask) over the image and checking if all the pixels in the element match the corresponding pixels in the image.
    #If there is at least one mismatch, the pixel in the center of the structuring element is set to 0 (black), otherwise it is set to 1 (white).
    #As a result, the foreground pixels in the input image that are not completely covered by the structuring element are removed in the output image.
    # The size of the foreground objects is thus reduced, while their shape is preserved as much as possible.
    #The process is repeated iteratively until the foreground objects can no longer be further reduced, or until a desired level of erosion has been achieved.
    mask_eroded = morphology.binary_erosion(mask, struct_el) #erodes the binary mask using the strucutring element, resulting in a new binary mask with the same shape as the original mask but with the edges erroded.
    image_perimeter = mask - mask_eroded  #subtracts the eroded mask from the original mask to obtain an image that contains only the perimeter of the lesion.
    perimeter = np.sum(image_perimeter) #caculates the perimeter of the lesion by summing the values of all pixels in the perimeter image.
    return [area, perimeter] 


In [ ]:
#training and testing data : trying to split the data set into training data and  testing data
# import matplotlib.pyplot as plot
# from sklearn.datasets import load_files

# data_folder_path = '/Users/nuvanjanashashipraba/Desktop/Images'
# my_data = load_files(data_folder_path, categories =['folder1', 'folder2', 'folder3']) # loading the data from the folder path specified in step 3 and categorizing them into tthree folders

# X = my_data.data #assigning the image data to X
# Y = my_data.target #assigning the target labels to Y

# from sklearn.model_selection import train_test_split #importing the train_test_split function from the sklearn.model _slection library for splitting the data into trainin and testing sets.
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 2)
# #splitting the data into training and testing sets, with 20% of the data set aside for testing and setting the random state for reproducibility.
#The resulting four variables represent the training and testing data for both the image data.

#print("The dimensions of the features of training data ")
#print(X_train.shape)
#print("The dimensions of the features of test data ")
#print(X_test.shape)
#print("The dimensions of the target values of training data ")
#print(Y_train.shape)
#print("The dimensions of the target values of test data ")
#print(Y_test.shape)

In [30]:
def knn_prediction(k, train, classes, test): #classes : the class labels of the training data.
    neighb = KNeighborsClassifier(n_neighbors= k)
    neighb.fit(train, classes.ravel())
    classes_pred = neighb.predict(test)
    return classes_pred

In [31]:
def accuracy_test(k,train,classes,test,classes_test):
    performance = []
    perform1 =[]
    #the 'performance' list stores the predicted lables for each value of k
    #the 'perform1' list stores the accuracy score for each value of k

    for i in range(1,k):
        a = make_knn_prediction(i,train,classes,test)
        performance.append(a)
        for j in performance:
            b = accuracy_score(classes_test,j) #'accuracy_score' is function from scikit learn 
        perform1.append(b)
    fig, axes = plt.subplots()
    axes.plot(perform1)
    plt.title("Classification Accuracy of KNN for Different Values of k")
    plt.ylabel("Test Accuracy")
    plt.xlabel("Value of k");
    plt.grid(color='grey', linestyle='--', linewidth=0.5)
    return np.mean(perform1)

### Splittting the data set for the color

In [62]:
#checking the path for the folder
import os

# Get the path to the images folder
folder_name = "processed_images"
folder_path = os.path.join(os.getcwd(), folder_name)

# Print the path
print(folder_path)


/Users/nuvanjanashashipraba/Documents/GitHub/cherry2/processed_images


In [64]:
from PIL import Image

color_data = []

# Loop through all the images in the folder and extract the number of colors
for filename in os.listdir(image):
    img = Image.open(os.path.join(image, filename))
    colors = img.getcolors()
    if colors:
        color_data.append(len(colors))
    
# Convert the list to a numpy array
color_data = np.array(color_data)

# Split the data into training and testing sets
color_train, color_test, clas_train, clas_test = train_test_split(color_data, clas, test_size=0.6, random_state=4)

# Reshape the training and testing data
color_train = color_train.reshape(-1, 1)
clas_train = clas_train.reshape(-1, 1)
color_test = color_test.reshape(-1, 1)


UnidentifiedImageError: cannot identify image file '/Users/nuvanjanashashipraba/Documents/GitHub/cherry2/processed_images/.DS_Store'

In [56]:
# color_data = my_data.to_numpy() # what is the df_features_extra in our case? what is the "Nb_of-colours" in our case?
# #df_features = the name of the csv or the varibale name that you used to read the csv file or the data set
# #because our data set is not a csv  file, can we just write: name of the dataset.to_numpy()

# color_train, color_test, clas_train, clas_test = train_test_split(color_data, clas, test_size = 0.6, random_state= 4)

# color_train = color_train. reshape(-1, 1)
# clas_train = clas_train.reshape (-1, 1)
# color_test = color_test.reshape( -1, 1)

In [ ]:
print(knn_prediction(7, color_train, clas_train, color_test))
print(clas_test)

In [ ]:
accuracy_test(380,color_train, clas_train, color_test, clas_test)

### Splitting the data set for the Saturation 

In [ ]:
sat_data = df_features_extra["sat_value"].to_numpy()
sat_train, sat_test, clas_train, clas_test = train_test_split(sat_data,clas,test_size=0.6, random_state=4)

sat_train = sat_train,reshape(-1,1)
clas_train = clas_train.reshape(-1,1)
sat_test = sat_test.reshape(-1,1)


In [ ]:
print(knn_prediction(7,sat_train, clas_train,sat_test))
print(clas_test)

In [ ]:
accuracy_test(150, sat_train,clas_train,sat_test,clas_test)


### Splitting the data set for the Asymmetry

In [ ]:
#Splittng the data set for the Asymmetry
aymmetry_data =df_features_extra ["Aymmetry"].to_numpy() #but what is the df_features_extra in our case? # what is the "Asymmetry" in our case.
#df_features = the name of the csv or the varibale name that you used to read the csv file or the data set
#because our data set is not a csv  file, can we just write: name of the dataset.to_numpy()
asymmetry_train = asymmetry_test, clas_train, clas_test = train_test_split(asymmetry_data, clas, test_size = 0.6, random_state= 4)
asymmetry_train = asymmetry_train.reshape(-1,1)
clas_train = clas_train.reshape(-1, 1)
asymmetry_test = asymmetry_test.reshape(-1,1)

In [ ]:
print(knn_prediction(7,asymmetry_train, clas_train,asymmetry_test))
print(clas_test)


In [ ]:
accuracy_test(800,asymmetry_train,clas_train,asymmetry_test.clas_test)
